# Object Detection

## Import everything

In [3]:
import os
import yaml
import sys

from ote_cli.common import (add_hyper_parameters_sub_parser, create_project,
                            gen_params_dict_from_args, get_task_impl_class,
                            load_config, load_model_weights)
from ote_cli.datasets import get_dataset_class
from ote_cli.registry.registry import Registry
from sc_sdk.entities.analyse_parameters import AnalyseParameters
from sc_sdk.entities.datasets import NullDataset, Subset
from sc_sdk.entities.model import Model
from sc_sdk.entities.resultset import ResultSet
from sc_sdk.entities.task_environment import TaskEnvironment

## Register templates

In [4]:
templates_dir = '../../model_templates'
registry = Registry(templates_dir)
registry = registry.filter(framework=sys.executable.split(os.sep)[-4])

print(registry)

- domain: Object_Detection
  framework: OTEDetection_v2.9.1
  name: Custom_Object_Detection_Gen2_ATSS
  path: /home/ikrylov/git/openvino_training_extensions/model_templates/OTEDetection_v2.9.1/Object_Detection/Custom_Object_Detection_Gen2_ATSS/template.yaml
- domain: Object_Detection
  framework: OTEDetection_v2.9.1
  name: Custom_Object_Detection_Gen2_SSD
  path: /home/ikrylov/git/openvino_training_extensions/model_templates/OTEDetection_v2.9.1/Object_Detection/Custom_Object_Detection_Gen2_SSD/template.yaml
- domain: Object_Detection
  framework: OTEDetection_v2.9.1
  name: Custom_Object_Detection_Gen2_VFNet
  path: /home/ikrylov/git/openvino_training_extensions/model_templates/OTEDetection_v2.9.1/Object_Detection/Custom_Object_Detection_Gen2_VFNet/template.yaml
- domain: Object_Detection
  framework: OTEDetection_v2.9.1
  name: Custom_Object_Detection_256x256
  path: /home/ikrylov/git/openvino_training_extensions/model_templates/OTEDetection_v2.9.1/Object_Detection/Custom_Object_Dete

## Load model template

In [5]:
template = registry.get('Custom_Object_Detection_256x256')
config = load_config(template['path'])
config['hyper_parameters']['params'].setdefault('algo_backend', {})['template'] = os.path.abspath(template['path'])

## Get training task and dataset instantiated

In [6]:
Task = get_task_impl_class(config)
Dataset = get_dataset_class(config['domain'])

dataset_paths = {
    'train_ann_file' : '../../data/airport/annotation_faces_train.json',
    'train_data_root' : '../../data/airport/',
    'val_ann_file' : '../../data/airport/annotation_faces_train.json',
    'val_data_root' : '../../data/airport/',
}

dataset = Dataset(**dataset_paths)

project = create_project(dataset.get_labels())
environment = TaskEnvironment(project=project, task_node=project.tasks[-1])
params = Task.get_configurable_parameters(environment)
Task.apply_template_configurable_parameters(params, config)
environment.set_configurable_parameters(params)
task = Task(task_environment=environment)

dataset.set_project_labels(project.get_labels())

ModuleNotFoundError: No module named 'sc_sdk.usecases.tasks.interfaces.configurable_parameters_interface'

## Have a look at existing parameters

In [ ]:
params = task.get_configurable_parameters(environment)
yaml.dump(params.learning_parameters, sys.stdout)

## Tweak parameters

In [ ]:
params.learning_parameters.learning_rate.value = 1e-3
params.learning_parameters.learning_rate_schedule.value = 'cyclic'
params.learning_parameters.batch_size.value = 2
params.learning_parameters.num_epochs.value = 10

environment.set_configurable_parameters(params)
task.update_configurable_parameters(environment)

## Train a model

In [ ]:
model = task.train(dataset)

## Evaluate quality metric

In [ ]:
validation_dataset = dataset.get_subset(Subset.VALIDATION)
predicted_validation_dataset = task.analyse(
    validation_dataset.with_empty_annotations(),
    AnalyseParameters(is_evaluation=True))

resultset = ResultSet(
    model=model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)

performance = task.compute_performance(resultset)
resultset.performance = performance